In [1]:
import numpy as np
import pandas as pd
from dstk.utils.data_cleaning import clean_columns

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

sns.set_style('dark')
plt.rcParams['figure.figsize'] = (9,6)
np.set_printoptions(suppress=True)
pd.options.display.max_columns = 150
pd.options.display.float_format =  lambda x: '{:,.3f}'.format(x)

In [2]:
def col_descrip(table, col):
    "Looks up column description for given table"
    print(table)
    print(col)
    return col_des.loc[(col_des.Table==table) & (col_des.Row==col.upper()),
            'Description'].values[0]

def anom_eval(df, col, filename):
    print(col_descrip(filename, col))
    print()

    max_diff_idx = df[col].sort_values().diff().nlargest(3).index
    max_diffs = df.loc[max_diff_idx, col]
    nlargest = df[col].nlargest()
    nsmallest = df[col].nsmallest()

    print('Max Diffs')
    print(max_diffs)
    print()
    print("Largest Vals")
    print(nlargest)
    print()
    print("Smallest Vals")
    print(nsmallest)
    df[col].hist()
    df[col].value_counts().sort_index()

    return max_diffs, nlargest

def pct_null(df):
    null_counts = df.isnull().sum()[df.isnull().sum()>0].sort_values(ascending=False).to_frame()
    if null_counts.empty:
        return None
    null_counts['pct_null'] = null_counts/df.shape[0]
    null_counts.columns = ['n_null', 'pct_null']
    return null_counts

In [3]:
col_des = pd.read_csv('HomeCredit_columns_description.csv', encoding='latin-1')
col_des.shape

(219, 5)

Read all of dfs

Gonna have to agg more to join to applications data

In [4]:
dtype={'sk_id_curr':str, 'sk_id_bureau':str, 'sk_id_prev':str,'num_instalment_version':str}

In [5]:
app_train = pd.read_csv('clean_data/app_train.csv', dtype=dtype)
app_test = pd.read_csv('clean_data/app_test.csv', dtype=dtype)

bureau = pd.read_csv('clean_data/bureau_mrgd.csv', dtype=dtype)
prev_apps_agg = pd.read_csv('clean_data/prev_apps.csv', dtype=dtype)
cc_agg = pd.read_csv('clean_data/cc_agg.csv', dtype=dtype)
pos_agg = pd.read_csv('clean_data/pos_agg.csv', dtype=dtype)
payments_agg = pd.read_csv('clean_data/payments_agg.csv', dtype=dtype)

In [6]:
app_train.head()

,sk_id_curr,target,flag_own_car,flag_own_realty,name_contract_type,flag_cont_mobile,flag_document_10,flag_document_11,flag_document_12,flag_document_13,flag_document_14,flag_document_15,flag_document_16,flag_document_17,flag_document_18,flag_document_19,flag_document_2,flag_document_20,flag_document_21,flag_document_3,flag_document_4,flag_document_5,flag_document_6,flag_document_7,flag_document_8,flag_document_9,flag_email,flag_emp_phone,flag_mobil,flag_phone,flag_work_phone,live_city_not_work_city,live_region_not_work_region,reg_city_not_live_city,reg_city_not_work_city,reg_region_not_live_region,reg_region_not_work_region,cnt_children,days_birth,days_employed,days_id_publish,hour_appr_process_start,region_rating_client,region_rating_client_w_city,amt_annuity,amt_credit,amt_goods_price,amt_income_total,amt_req_credit_bureau_day,amt_req_credit_bureau_hour,amt_req_credit_bureau_mon,amt_req_credit_bureau_qrt,amt_req_credit_bureau_week,amt_req_credit_bureau_year,apartments_avg,apartments_medi,apartments_mode,basementarea_avg,basementarea_medi,basementarea_mode,cnt_fam_members,commonarea_avg,commonarea_medi,commonarea_mode,days_last_phone_change,days_registration,def_30_cnt_social_circle,def_60_cnt_social_circle,elevators_avg,elevators_medi,elevators_mode,entrances_avg,entrances_medi,entrances_mode,ext_source_1,...,weekday_appr_process_start_TUESDAY,weekday_appr_process_start_WEDNESDAY,organization_type_Agriculture,organization_type_Bank,organization_type_Business Entity Type 1,organization_type_Business Entity Type 2,organization_type_Business Entity Type 3,organization_type_Cleaning,organization_type_Construction,organization_type_Culture,organization_type_Electricity,organization_type_Emergency,organization_type_Government,organization_type_Hotel,organization_type_Housing,organization_type_Industry: type 1,organization_type_Industry: type 10,organization_type_Industry: type 11,organization_type_Industry: type 12,organization_type_Industry: type 13,organization_type_Industry: type 2,organization_type_Industry: type 3,organization_type_Industry: type 4,organization_type_Industry: type 5,organization_type_Industry: type 6,organization_type_Industry: type 7,organization_type_Industry: type 8,organization_type_Industry: type 9,organization_type_Insurance,organization_type_Kindergarten,organization_type_Legal Services,organization_type_Medicine,organization_type_Military,organization_type_Mobile,organization_type_Other,organization_type_Police,organization_type_Postal,organization_type_Realtor,organization_type_Religion,organization_type_Restaurant,organization_type_School,organization_type_Security,organization_type_Security Ministries,organization_type_Self-employed,organization_type_Services,organization_type_Telecom,organization_type_Trade: type 1,organization_type_Trade: type 2,organization_type_Trade: type 3,organization_type_Trade: type 4,organization_type_Trade: type 5,organization_type_Trade: type 6,organization_type_Trade: type 7,organization_type_Transport: type 1,organization_type_Transport: type 2,organization_type_Transport: type 3,organization_type_Transport: type 4,organization_type_University,organization_type_XNA,fondkapremont_mode_not specified,fondkapremont_mode_org spec account,fondkapremont_mode_reg oper account,fondkapremont_mode_reg oper spec account,housetype_mode_block of flats,housetype_mode_specific housing,housetype_mode_terraced house,wallsmaterial_mode_MISSING,wallsmaterial_mode_Mixed,wallsmaterial_mode_Monolithic,wallsmaterial_mode_Others,wallsmaterial_mode_Panel,"wallsmaterial_mode_Stone, brick",wallsmaterial_mode_Wooden,emergencystate_mode_No,emergencystate_mode_Yes
0,100002,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0.000,9461,637.000,2120,10,2,2,"24,700.500","406,597.500","351,000.000","202,500.000",0.000,0.000,0.000,0.000,0.000,1.000,0.025,0.025,0.025,0.037,0.037,0.038,1.000,0.014,0.014,0.014,"1,134.000","3,648.000",2.000,2.000,0.000,0.000,0.000,0.069,0.069,0.069,0.083

#### Agg Bureau and Join

In [7]:
bureau.head()

,sk_id_bureau,sk_id_curr,cnt_credit_prolong,credit_day_overdue,days_credit,days_credit_update,amt_annuity,amt_credit_max_overdue,amt_credit_sum,amt_credit_sum_debt,amt_credit_sum_limit,amt_credit_sum_overdue,days_credit_enddate,days_enddate_fact,days_credit_update_anom,days_credit_enddate_anom,days_enddate_fact_anom,amt_credit_max_overdue_anom,amt_annuity_anom,amt_credit_sum_anom,amt_credit_sum_debt_anom,amt_credit_sum_limit_anom,amt_credit_sum_overdue_anom,credit_active_Bad debt,credit_active_Closed,credit_active_Sold,credit_currency_currency 2,credit_currency_currency 3,credit_currency_currency 4,credit_type_Car loan,credit_type_Cash loan (non-earmarked),credit_type_Consumer credit,credit_type_Credit card,credit_type_Interbank credit,credit_type_Loan for business development,credit_type_Loan for purchase of shares (margin lending),credit_type_Loan for the purchase of equipment,credit_type_Loan for working capital replenishment,credit_type_Microloan,credit_type_Mobile operator loan,credit_type_Mortgage,credit_type_Real estate loan,credit_type_Unknown type of loan,balance_status_0_lt1yr,balance_status_0_yr1_5,balance_status_0_yr5_10,balance_status_0_gt10yr,balance_status_1_lt1yr,balance_status_1_yr1_5,balance_status_1_yr5_10,balance_status_1_gt10yr,balance_status_2_lt1yr,balance_status_2_yr1_5,balance_status_2_yr5_10,balance_status_2_gt10yr,balance_status_3_lt1yr,balance_status_3_yr1_5,balance_status_3_yr5_10,balance_status_3_gt10yr,balance_status_4_lt1yr,balance_status_4_yr1_5,balance_status_4_yr5_10,balance_status_4_gt10yr,balance_status_5_lt1yr,balance_status_5_yr1_5,balance_status_5_yr5_10,balance_status_5_gt10yr,balance_status_C_lt1yr,balance_status_C_yr1_5,balance_status_C_yr5_10,balance_status_C_gt10yr
0,5714462,215354,0,0,497,131.000,0.000,0.000,"91,323.000",0.000,0.000,0.000,153.000,153.000,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.000,3.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1,5714463,215354,0,0,208,20.000,0.000,0.000,"225,000.000","171,342.000",0.000,0.000,"1,075.000",897.000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0.000,3.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2,5714464,215354,0,0,203,16.000,0.000,0.000,"464,323.500",0.000,0.000,0.000,528.000,897.000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.000,3.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
3,5714465,215354,0,0,203,16.000,0.000,0.000,"90,000.000",0.000,0.000,0.000,862.000,897.000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0.000,3.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
4,5714466,215354,0,0,629,21.000,0.000,"77,674.500","2,700,000.000",0.000,0.000,0.000,"1,197.000",897.000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.000,3.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [8]:
bureau.columns.tolist().index('days_credit_update_anom')

14

In [9]:
bureau.shape

(1716428, 71)

In [10]:
agg_dic = dict.fromkeys(bureau.select_dtypes('number').columns.tolist(), 'mean')
sum_cols = bureau.columns[14:]
for s in sum_cols:
    agg_dic[s] = 'sum'
    
bureau_agg = bureau.groupby('sk_id_curr').agg(agg_dic).reset_index()
bureau_agg.shape


(305811, 70)

In [11]:
bureau_agg.columns[bureau_agg.columns.isin(app_train.columns)]

Index(['sk_id_curr', 'amt_annuity'], dtype='object')

In [12]:
print(app_train.shape, bureau_agg.shape)
mrgd_train = app_train.merge(bureau_agg, 'left', 'sk_id_curr', suffixes=['_application', '_bureau'])
mrgd_test = app_test.merge(bureau_agg, 'left', 'sk_id_curr', suffixes=['_application', '_bureau'])
print(mrgd_train.shape, mrgd_test.shape)

(307511, 244) (305811, 70)
(307511, 313) (48744, 309)


#### Prev Apps

In [13]:
prev_apps_agg.drop('flag_last_appl_per_contract', axis=1, inplace=True)
prev_apps_agg.head()


,sk_id_curr,sk_id_prev,days_decision,hour_appr_process_start,sellerplace_area,amt_annuity,amt_application,amt_credit,amt_down_payment,amt_goods_price,cnt_payment,days_first_due,days_last_due,days_last_due_1st_version,days_termination,rate_down_payment,rate_interest_primary,rate_interest_privileged,nflag_insured_on_approval,nflag_last_appl_in_day,sellerplace_area_anom,amt_annuity_anom,days_first_due_anom,days_last_due_1st_version_anom,days_last_due_anom,days_termination_anom,name_contract_type_Consumer loans,name_contract_type_Revolving loans,name_contract_type_XNA,weekday_appr_process_start_MONDAY,weekday_appr_process_start_SATURDAY,weekday_appr_process_start_SUNDAY,weekday_appr_process_start_THURSDAY,weekday_appr_process_start_TUESDAY,weekday_appr_process_start_WEDNESDAY,name_cash_loan_purpose_Business development,name_cash_loan_purpose_Buying a garage,name_cash_loan_purpose_Buying a holiday home / land,name_cash_loan_purpose_Buying a home,name_cash_loan_purpose_Buying a new car,name_cash_loan_purpose_Buying a used car,name_cash_loan_purpose_Car repairs,name_cash_loan_purpose_Education,name_cash_loan_purpose_Everyday expenses,name_cash_loan_purpose_Furniture,name_cash_loan_purpose_Gasification / water supply,name_cash_loan_purpose_Hobby,name_cash_loan_purpose_Journey,name_cash_loan_purpose_Medicine,name_cash_loan_purpose_Money for a third person,name_cash_loan_purpose_Other,name_cash_loan_purpose_Payments on other loans,name_cash_loan_purpose_Purchase of electronic equipment,name_cash_loan_purpose_Refusal to name the goal,name_cash_loan_purpose_Repairs,name_cash_loan_purpose_Urgent needs,name_cash_loan_purpose_Wedding / gift / holiday,name_cash_loan_purpose_XAP,name_cash_loan_purpose_XNA,name_contract_status_Canceled,name_contract_status_Refused,name_contract_status_Unused offer,name_payment_type_Cashless from the account of the employer,name_payment_type_Non-cash from your account,name_payment_type_XNA,code_reject_reason_HC,code_reject_reason_LIMIT,code_reject_reason_SCO,code_reject_reason_SCOFR,code_reject_reason_SYSTEM,code_reject_reason_VERIF,code_reject_reason_XAP,code_reject_reason_XNA,name_type_suite_Family,name_type_suite_Group of people,...,name_type_suite_Unaccompanied,name_type_suite_nan,name_client_type_Refreshed,name_client_type_Repeater,name_client_type_XNA,name_goods_category_Animals,name_goods_category_Audio/Video,name_goods_category_Auto Accessories,name_goods_category_Clothing and Accessories,name_goods_category_Computers,name_goods_category_Construction Materials,name_goods_category_Consumer Electronics,name_goods_category_Direct Sales,name_goods_category_Education,name_goods_category_Fitness,name_goods_category_Furniture,name_goods_category_Gardening,name_goods_category_Homewares,name_goods_category_House Construction,name_goods_category_Insurance,name_goods_category_Jewelry,name_goods_category_Medical Supplies,name_goods_category_Medicine,name_goods_category_Mobile,name_goods_category_Office Appliances,name_goods_category_Other,name_goods_category_Photo / Cinema Equipment,name_goods_category_Sport and Leisure,name_goods_category_Tourism,name_goods_category_Vehicles,name_goods_category_Weapon,name_goods_category_XNA,name_portfolio_Cars,name_portfolio_Cash,name_portfolio_POS,name_portfolio_XNA,name_product_type_walk-in,name_product_type_x-sell,channel_type_Car dealer,channel_type_Channel of corporate sales,channel_type_Contact center,channel_type_Country-wide,channel_type_Credit and cash offices,channel_type_Regional / Local,channel_type_Stone,name_seller_industry_Clothing,name_seller_industry_Connectivity,name_seller_industry_Construction,name_seller_industry_Consumer electronics,name_seller_industry_Furniture,name_seller_industry_Industry,name_seller_industry_Jewelry,name_seller_industry_MLM partners,name_seller_industry_Tourism,name_seller_industry_XNA,name_yield_group_high,name_yield_group_low_action,name_yield_group_low_normal,name_yield_group_middle,product_combination_Card X-Sell,product_combination_

In [14]:
agg_dic = dict.fromkeys(prev_apps_agg.select_dtypes('number').columns.tolist(), 'mean')
sum_cols = prev_apps_agg.columns[19:].tolist()
prev_apps_agg.loc[:,sum_cols] = prev_apps_agg[sum_cols].fillna(0)

for s in sum_cols:
    agg_dic[s] = 'sum'

    
prev_apps_agg = prev_apps_agg.groupby('sk_id_curr').agg(agg_dic).reset_index()
prev_apps_agg.shape

(338857, 152)

In [15]:
prev_apps_agg.columns[prev_apps_agg.columns.isin(mrgd_train.columns)]

Index(['sk_id_curr', 'hour_appr_process_start', 'amt_credit',
       'amt_goods_price', 'amt_annuity_anom',
       'weekday_appr_process_start_MONDAY',
       'weekday_appr_process_start_SATURDAY',
       'weekday_appr_process_start_SUNDAY',
       'weekday_appr_process_start_THURSDAY',
       'weekday_appr_process_start_TUESDAY',
       'weekday_appr_process_start_WEDNESDAY', 'name_type_suite_Family',
       'name_type_suite_Group of people', 'name_type_suite_Other_A',
       'name_type_suite_Other_B', 'name_type_suite_Spouse, partner',
       'name_type_suite_Unaccompanied'],
      dtype='object')

In [16]:
print(mrgd_train.shape, prev_apps_agg.shape)
mrgd_train = mrgd_train.merge(prev_apps_agg, 'left', 'sk_id_curr', suffixes=['_application', '_prev_apps'])
mrgd_test = mrgd_test.merge(prev_apps_agg, 'left', 'sk_id_curr', suffixes=['_application', '_prev_apps'])
mrgd_train.shape

(307511, 313) (338857, 152)


(307511, 464)

#### Credit Cards

In [17]:
cc_agg.head()

,sk_id_curr,sk_id_prev,amt_credit_limit_actual,cnt_drawings_current,months_balance,sk_dpd,amt_balance,amt_drawings_atm_current,amt_drawings_current,amt_drawings_other_current,amt_drawings_pos_current,amt_inst_min_regularity,amt_payment_current,amt_payment_total_current,amt_receivable_principal,amt_recivable,amt_total_receivable,cnt_drawings_atm_current,cnt_drawings_other_current,cnt_drawings_pos_current,cnt_instalment_mature_cum,amt_payment_current_anom,amt_payment_total_current_anom,cnt_drawings_atm_current_anom,name_contract_status_Approved,name_contract_status_Completed,name_contract_status_Demand,name_contract_status_Refused,name_contract_status_Sent proposal,name_contract_status_Signed
0,100006,1489396,"270,000.000",0.000,3.500,0.000,0.000,0.000,0.000,0.000,0.000,0.000,"2,702.700",0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0,0,0,0,0,0,0,0,0
1,100011,1843384,"164,189.189",0.054,38.500,0.000,"54,482.111","2,432.432","2,432.432",0.000,0.000,"3,902.759","4,843.064","4,520.068","52,402.089","54,433.179","54,433.179",0.054,0.000,0.000,25.622,0,0,0,0,0,0,0,0,0
2,100013,2038692,"131,718.750",0.240,48.500,0.010,"18,159.919","5,953.125","5,953.125",0.000,0.000,"1,348.479","7,168.346","6,817.172","17,255.560","18,101.080","18,101.080",0.240,0.000,0.000,18.448,0,0,0,0,0,0,0,0,0
3,100021,2594025,"675,000.000",0.000,10.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,"2,702.700",0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0,0,0,0,10,0,0,0,0
4,100023,1499902,"135,000.000",0.000,7.500,0.000,0.000,0.000,0.000,0.000,0.000,0.000,"2,702.700",0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0,0,0,0,0,0,0,0,0


In [18]:
agg_dic = dict.fromkeys(cc_agg.select_dtypes('number').columns.tolist(), 'mean')
sum_cols = cc_agg.columns[-9:].tolist()

for s in sum_cols:
    agg_dic[s] = 'sum'

    
cc_agg = cc_agg.groupby('sk_id_curr').agg(agg_dic).reset_index()
cc_agg.shape

(103558, 29)

In [19]:
cc_agg.columns[cc_agg.columns.isin(mrgd_train.columns)]

Index(['sk_id_curr', 'name_contract_status_Refused'], dtype='object')

In [20]:
print(mrgd_train.shape, cc_agg.shape)
mrgd_train = mrgd_train.merge(cc_agg, 'left', 'sk_id_curr', suffixes=['_application', '_credit_cards'])
mrgd_test = mrgd_test.merge(cc_agg, 'left', 'sk_id_curr', suffixes=['_application', '_credit_cards'])
mrgd_train.shape

(307511, 464) (103558, 29)


(307511, 492)

#### Installments Payments

In [21]:
payments_agg.head()

,sk_id_curr,sk_id_prev,amt_instalment,amt_payment,days_entry_payment,days_instalment,days_entry_payment_anom,amt_instalment_anom,amt_payment_anom,num_instalment_version_1.0,num_instalment_version_10.0,num_instalment_version_11.0,num_instalment_version_12.0,num_instalment_version_13.0,num_instalment_version_14.0,num_instalment_version_15.0,num_instalment_version_16.0,num_instalment_version_17.0,num_instalment_version_178.0,num_instalment_version_18.0,num_instalment_version_19.0,num_instalment_version_2.0,num_instalment_version_20.0,num_instalment_version_21.0,num_instalment_version_22.0,num_instalment_version_23.0,num_instalment_version_24.0,num_instalment_version_25.0,num_instalment_version_26.0,num_instalment_version_27.0,num_instalment_version_28.0,num_instalment_version_29.0,num_instalment_version_3.0,num_instalment_version_30.0,num_instalment_version_31.0,num_instalment_version_32.0,num_instalment_version_33.0,num_instalment_version_34.0,num_instalment_version_35.0,num_instalment_version_36.0,num_instalment_version_37.0,num_instalment_version_38.0,num_instalment_version_39.0,num_instalment_version_4.0,num_instalment_version_40.0,num_instalment_version_41.0,num_instalment_version_42.0,num_instalment_version_43.0,num_instalment_version_44.0,num_instalment_version_45.0,num_instalment_version_46.0,num_instalment_version_47.0,num_instalment_version_48.0,num_instalment_version_49.0,num_instalment_version_5.0,num_instalment_version_50.0,num_instalment_version_51.0,num_instalment_version_52.0,num_instalment_version_53.0,num_instalment_version_54.0,num_instalment_version_55.0,num_instalment_version_56.0,num_instalment_version_57.0,num_instalment_version_58.0,num_instalment_version_59.0,num_instalment_version_6.0,num_instalment_version_61.0,num_instalment_version_68.0,num_instalment_version_7.0,num_instalment_version_72.0,num_instalment_version_73.0,num_instalment_version_8.0,num_instalment_version_9.0
0,100001,1369693,"29,250.900","29,250.900","6,718.000","1,664.000",0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,100001,1851984,"11,945.025","11,945.025","8,647.000","2,886.000",0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,100002,1038818,"219,625.695","219,625.695","5,993.000",295.000,0,0,0,18,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,100003,1810518,"1,150,977.330","1,150,977.330","4,413.000",626.000,0,0,0,6,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,100003,2396755,"80,773.380","80,773.380","25,821.000","2,145.000",0,0,0,12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [22]:
agg_dic = dict.fromkeys(payments_agg.select_dtypes('number').columns.tolist(), 'mean')
sum_cols = payments_agg.columns[6:].tolist()

for s in sum_cols:
    agg_dic[s] = 'sum'

    
payments_agg = payments_agg.groupby('sk_id_curr').agg(agg_dic).reset_index()
payments_agg.shape

(339587, 72)

In [23]:
payments_agg.columns[payments_agg.columns.isin(mrgd_train.columns)]

Index(['sk_id_curr'], dtype='object')

In [24]:
print(mrgd_train.shape, payments_agg.shape)
mrgd_train = mrgd_train.merge(payments_agg, 'left', 'sk_id_curr', suffixes=['_application', '_install_payments'])
mrgd_test = mrgd_test.merge(payments_agg, 'left', 'sk_id_curr', suffixes=['_application', '_install_payments'])
mrgd_train.shape

(307511, 492) (339587, 72)


(307511, 563)

#### POS

In [25]:
pos_agg.head()

,sk_id_prev,sk_id_curr,months_balance,sk_dpd,name_contract_status_Active,name_contract_status_Amortized debt,name_contract_status_Approved,name_contract_status_Canceled,name_contract_status_Completed,name_contract_status_Demand,name_contract_status_Returned to the store,name_contract_status_Signed
0,1000001,158271,9.000,0.000,2,0,0,0,1,0,0,0
1,1000002,101962,52.000,0.000,4,0,0,0,1,0,0,0
2,1000003,252457,2.500,0.000,4,0,0,0,0,0,0,0
3,1000004,260094,25.500,0.000,7,0,0,0,1,0,0,0
4,1000005,176456,51.000,0.000,10,0,0,0,1,0,0,0


In [26]:
agg_dic = dict.fromkeys(pos_agg.select_dtypes('number').columns.tolist(), 'mean')
sum_cols = pos_agg.columns[4:].tolist()

for s in sum_cols:
    agg_dic[s] = 'sum'

    
pos_agg = pos_agg.groupby('sk_id_curr').agg(agg_dic).reset_index()
pos_agg.shape

(337252, 11)

In [27]:
pos_agg.columns[pos_agg.columns.isin(mrgd_train.columns)]

Index(['sk_id_curr', 'months_balance', 'sk_dpd',
       'name_contract_status_Approved', 'name_contract_status_Canceled',
       'name_contract_status_Completed', 'name_contract_status_Demand',
       'name_contract_status_Signed'],
      dtype='object')

In [28]:
print(mrgd_train.shape, pos_agg.shape)
mrgd_train = mrgd_train.merge(pos_agg, 'left', 'sk_id_curr', suffixes=['_application', '_pos'])
mrgd_test = mrgd_test.merge(pos_agg, 'left', 'sk_id_curr', suffixes=['_application', '_pos'])
mrgd_train.shape

(307511, 563) (337252, 11)


(307511, 573)

### Align Train to Test

In [29]:
target = mrgd_train['target']
print(mrgd_train.shape, mrgd_test.shape)
mrgd_train, mrgd_test = mrgd_train.align(mrgd_test, 'right', 1)
mrgd_train['target'] = target
print(mrgd_train.shape, mrgd_test.shape)

(307511, 573) (48744, 569)
(307511, 570) (48744, 569)


### Nulls

In [30]:
train_nulls = pct_null(mrgd_train)
# drop columns with more than 50% null
drop_cols = train_nulls[train_nulls.pct_null >=0.5].index

mrgd_train.drop(drop_cols, axis=1, inplace=True)
mrgd_test.drop(drop_cols, axis=1, inplace=True)

In [31]:
fill_cols = train_nulls[train_nulls.pct_null<0.5].index

In [32]:
fill_vals = mrgd_train[fill_cols].median()

In [33]:
mrgd_train.fillna(fill_vals, inplace=True)
mrgd_test.fillna(fill_vals,inplace=True)

In [34]:
mrgd_train.isnull().sum().sum(), mrgd_test.isnull().sum().sum()

(0, 0)

In [35]:
mrgd_train.to_csv('clean_data/mrgd_train.csv', index=False)
mrgd_test.to_csv('clean_data/mrgd_test.csv', index=False)